In [1]:
import numpy as np
import pandas as pd

from tqdm import tqdm
from collections import Counter

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, LSTM, Conv1D, GlobalAveragePooling1D
from keras.callbacks import ModelCheckpoint

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import explained_variance_score

from matplotlib import pyplot as plt
from matplotlib import rcParams

import sys
import time

C:\Users\voldo\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\__init__.py:177: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [7]:
data_pd = pd.read_csv("../test_data/medical_data_cleaned/data.txt", delimiter="|", names="ID|param_0|param_1|param_2|param_3|param_4|param_5|param_6|param_7".split("|")).drop(columns=["ID"])

In [14]:
x_data  = pd.DataFrame()
for col_ in data_pd.columns:
    x_data[col_] = (data_pd[col_] - data_pd[col_].mean()) / data_pd[col_].std()

In [15]:
x_data.head()

,param_0,param_1,param_2,param_3,param_4,param_5,param_6,param_7
0,0.497943,1.664932,1.956727,1.118066,0.715652,-0.574660,-1.532654,-1.584931
1,-0.254504,0.986995,1.956727,1.705385,-0.663918,-0.574660,-1.532654,-1.584931
2,-0.254504,0.309059,1.956727,0.824406,1.299317,0.354636,-1.532654,-1.584931
3,0.121719,0.986995,1.956727,1.705385,1.352377,-0.574660,-1.532654,-1.584931
4,1.250391,2.342868,1.956727,1.705385,-0.770039,-0.574660,-1.532654,-1.584931


In [17]:
y_data = pd.get_dummies(pd.read_csv("../test_data/medical_data_cleaned/check.txt", names=["class"])["class"])

In [22]:
y_data

,0,1,2,3
0,1,0,0,0
1,1,0,0,0
2,1,0,0,0
3,1,0,0,0
4,1,0,0,0
...,...,...,...,...
131,0,0,0,1
132,0,0,0,1
133,0,0,0,1
134,0,0,0,1


In [26]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(20, activation='relu', input_shape=(8,)),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 20)                180       
                                                                 
 dense_9 (Dense)             (None, 20)                420       
                                                                 
 dense_10 (Dense)            (None, 20)                420       
                                                                 
 dense_11 (Dense)            (None, 4)                 84        
                                                                 
Total params: 1,104
Trainable params: 1,104
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.compile(loss="categorical_crossentropy",
              metrics=["accuracy",], optimizer="adam")

In [28]:
model.fit(x_data, y_data, epochs=500)

Epoch 1/500
5/5 [==============================] - 1s 6ms/step - loss: 1.3877 - accuracy: 0.1912
Epoch 2/500
5/5 [==============================] - 0s 6ms/step - loss: 1.3232 - accuracy: 0.2574
Epoch 3/500
5/5 [==============================] - 0s 7ms/step - loss: 1.2694 - accuracy: 0.2647
Epoch 4/500
5/5 [==============================] - 0s 5ms/step - loss: 1.2264 - accuracy: 0.2647
Epoch 5/500
5/5 [==============================] - 0s 6ms/step - loss: 1.1840 - accuracy: 0.2941
Epoch 6/500
5/5 [==============================] - 0s 4ms/step - loss: 1.1494 - accuracy: 0.3750
Epoch 7/500
5/5 [==============================] - 0s 6ms/step - loss: 1.1201 - accuracy: 0.4191
Epoch 8/500
5/5 [==============================] - 0s 7ms/step - loss: 1.0949 - accuracy: 0.4706
Epoch 9/500
5/5 [==============================] - 0s 5ms/step - loss: 1.0747 - accuracy: 0.5000
Epoch 10/500
5/5 [==============================] - 0s 4ms/step - loss: 1.0563 - accuracy: 0.5074
Epoch 11/500
5/5 [===========

# Sensitivity analysis (or parameter informativeness)

# Permutation Feature Importance:

This method involves permuting the values of a single feature and observing the impact on the model's performance. The drop in performance indicates the importance of that feature.

In [75]:
from sklearn.metrics import accuracy_score

def permutation_feature_importance(model_, x, y, metric=accuracy_score):
    x = np.array(x)
    y = np.array(y)
    baseline_score = metric(np.array([np.argmax(rec_) for rec_ in y]),
                            np.array([np.argmax(rec_) for rec_ in np.array(model_.predict(x, verbose=0))]))

    importance_scores_ = []
    for i in range(x.shape[1]):
        x_permuted = x.copy()
        np.random.shuffle(x_permuted[:, i])
        permuted_score = metric(
            np.array([np.argmax(rec_) for rec_ in y]),
            np.array([np.argmax(rec_) for rec_ in np.array(model_.predict(x_permuted, verbose=0))])
        )
        importance_scores_.append(baseline_score - permuted_score)

    return importance_scores_

# Example usage
importance_scores = permutation_feature_importance(model, x_data, y_data)

In [76]:
importance_scores

[0.26470588235294124,
 0.24264705882352944,
 0.05147058823529416,
 0.2279411764705883,
 0.1911764705882354,
 0.10294117647058831,
 0.30147058823529416,
 0.20588235294117652]

# Integrated Gradients:

Integrated Gradients is a technique that assigns an importance score to each feature by integrating the gradients of the model's output with respect to the input along a straight path from a baseline input to the actual input.

In [ ]:
from integrated_gradients import integrated_gradients

# Assuming 'integrated_gradients' is a module implementing the method
ig = integrated_gradients(model)
importance_scores = ig.explain(x_test)

# Gradient-based Methods:

You can use gradient-based methods to visualize the impact of input features on the loss. By calculating the gradients of the output with respect to the input features, you can identify which features have a larger influence on the loss.

In [78]:
loss_function = tf.keras.losses.categorical_crossentropy

with tf.GradientTape() as tape:
    inputs = tf.constant(x_data, dtype=tf.float32)
    tape.watch(inputs)
    predictions = model(inputs)
    loss = loss_function(y_data, predictions)

gradients = tape.gradient(loss, inputs)
feature_importance = tf.reduce_mean(tf.abs(gradients), axis=0)

In [79]:
feature_importance

<tf.Tensor: shape=(8,), dtype=float32, numpy=
array([0.17135061, 0.2093416 , 0.0922119 , 0.26909772, 0.14151053,
       0.32001853, 0.16301492, 0.35516405], dtype=float32)>

# Inspecting Weights:

For linear models, such as those created using tf.keras.layers.Dense, you can inspect the weights associated with each input feature. Larger absolute weights generally indicate a stronger influence on the model's output.

In [77]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, input_shape=(num_features,), activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Train the model

# Get the weights of the first layer
first_layer_weights = model.layers[0].get_weights()[0]


NameError: name 'num_features' is not defined